In [4]:
import requests
import numpy as np
import glob
import xlrd
import os
import sqlite3
from runSQL import *

def getFilesNames ():
    print("..................................................")
    print("Getting the name of the ANH files from the webpage")
    print("..................................................")
    url = 'http://www.anh.gov.co/estadisticas-del-sector/sistemas-integrados-operaciones/estad%C3%ADsticas-producci%C3%B3n'
    
    html = requests.get(url)
    with open('file.txt', 'wb') as wr: 
        wr.write(html.content)
    wr.close()

    read = open("file.txt","r+",encoding="utf-8")

    htmlExcel = ""
    for line in read:
        xlsx = ".xlsx"
        if xlsx in line:
            htmlExcel = line

    htmlExcel.replace(">", " ")
    htmlExcel.replace("<", " ")
    listaExcel = []

    separacionExcel= np.array(htmlExcel.split(" "))
    for i in range(0,len(separacionExcel)):
        if "href" in separacionExcel[i]:
            l = separacionExcel[i].split("\"")
            listaExcel.append("http://www.anh.gov.co"+l[1])
    read.close()
    os.remove('file.txt')
    return listaExcel

In [5]:
def downloadFiles (listaExcel):
    print("downloading files... this might take some minutes")
    for i in range(0, len(listaExcel)-1):
        dls = listaExcel[i]
        resp = requests.get(dls)
        name = dls.split("/")
        nombre = ""
        for j in range(0, len(name)):
            if ".xls" in name[j] or ".XLS" in name[j]:
                nombre = name[j].split(".xls")[0]
                nombre = nombre.replace("%c3", "on")
                nombre = nombre.replace("b3n%", "")
                nombre = nombre.replace("%","-")
                nombre = nombre.replace(".","")
                nombre = nombre.replace("2020","20")

                if len(name[j].split(".xls"))< 2:
                    nombre = nombre + ".xlsx"
                else:
                    if "x" in name[j].split(".xls")[1]:
                        nombre = nombre + ".xlsx"
                    else:
                        nombre = nombre + ".xls"
        
        output = open(nombre, 'wb')
        output.write(resp.content)
        output.close()
    print("Download complete!")
        

In [6]:
def getGasOilFileNames(result):
    local = ""
    if result == False:
        local = "Files\\"
    directorio = local+"*.xlsx"
    directorio2 = local + "*.xls"

    listnamefiles = [f for f in glob.glob(directorio)]
    listnamefiles2 = [f for f in glob.glob(directorio2)]
    listnameGas=[]
    listnameOil=[]
    listnameAll = []

    for i in range(0,len(listnamefiles)):
        if "Gas" in listnamefiles[i] or "gas" in listnamefiles[i]:
            listnameGas.append(listnamefiles[i])
        elif "Crudo" in listnamefiles[i] or "crudo" in listnamefiles[i]:
            listnameOil.append(listnamefiles[i])
        listnameAll.append(listnamefiles[i])
        
    for i in range(0,len(listnamefiles2)):
        if "Gas" in listnamefiles2[i] or "gas" in listnamefiles2[i]:
            listnameGas.append(listnamefiles2[i])
        elif "Crudo" in listnamefiles2[i] or "crudo" in listnamefiles2[i]:
            listnameOil.append(listnamefiles2[i])
        listnameAll.append(listnamefiles2[i])
    return listnameGas, listnameOil, listnameAll 


In [7]:


def encontrarEncabezado(sheet):
    
    encabezado = []
    indiceE = 0
    for i in range(0, 100):
        if sheet.row_values(i)[0]!=''and sheet.row_values(i)[1]!='':
            indiceE = i
            break
    for i in range(0, len(sheet.row_values(indiceE))):
        t = sheet.row_values(indiceE)
       
        if t[i] == '':
            break
        else:
            encabezado.append(t[i])
    
    return encabezado, indiceE
    
    
def encontrarAnio(sheet, indiceE):
    
    encontrado = False
    i = 0
    j = 0
    valor= ""
    while encontrado == False:

        valor = sheet.row_values(i)[j]
        if "20" in valor:
            valor = valor.split("20")[1]
            valor = "20"+valor
            valor = valor.replace("*","")
            if valor == "20":
                valor = "2020"
            encontrado= True
            break
        i+=1
        if i == indiceE:
            j+=1
            i=0
            
    return valor

def orgazinarLosArchivosOil(listnameOil):
    listAnio = []
    listEncabezado = []
    for i in range(0, len(listnameOil)):
        loc = listnameOil[i]
        wb = xlrd.open_workbook(loc)
        sheet = wb.sheet_by_index(0)
        sheet.cell_value(0, 0)

        encabezado, indiceE = encontrarEncabezado(sheet)
        anio = encontrarAnio(sheet, indiceE)
        siHay = 0
        for j in range(0, len(listAnio)):
            if anio == listAnio[j][0]:
                if len(encabezado)> len(listEncabezado[j][0]):
                    listAnio[j] = [anio,i]
                    listEncabezado[j] = [encabezado, indiceE]
                siHay = 1
        if siHay == 0:
            listAnio.append([anio, i])
            listEncabezado.append([encabezado, indiceE])
    return listAnio, listEncabezado

def guardarDatosOil(listnameOil):
    anio, encabezado = orgazinarLosArchivosOil(listnameOil)
    print("starting to save the data in the database this might take some minutes")
    for i in range(0,len(anio)):
        indice = anio[i][1]
        year = anio[i][0]
        loc = listnameOil[indice]
        print("saving: ",loc)
        wb = xlrd.open_workbook(loc)
        sheet = wb.sheet_by_index(0)
        sheet.cell_value(0, 0)
      
        for m in range(encabezado[i][1]+1, sheet.nrows):
            valor = sheet.row_values(m)
           
            if valor[0] == "" or valor[1] == "" or valor[2] == "" or "total" in valor[0].lower():
                break
            insertarTablaCrudo(valor,encabezado[i][0], anio[i][0])
        print("finish with: ", loc)

In [8]:
def checkifwork():
    listnamefiles = [f for f in glob.glob("*.xlsx")]
    listnamefiles2 = [f for f in glob.glob("*.xls")]

    number = len(listnamefiles) + len(listnamefiles2)
    print(number,"s")
    print("please enter the number of Excelfiles there are in github, in order to check and save in the database")
    x = input()
    result = False
    if number == int(x):
        print("the download was a success")
        result = True
    else:
        print("error downloading the files from github, please download the files manually and put them in the same folder of this file ")
        for i in range(0,len(listnamefiles)):
            os.remove(listnamefiles[i])
        for i in range(0,len(listnamefiles2)):
            os.remove(listnamefiles2[i])
    return result

In [9]:
def deleteFiles(listname):
    for i in range(0, len(listname)):
        os.remove(listname[i])


In [10]:
def getFilesNamesGitHub ():
    url = 'https://github.com/dgguarin20/archivos'
    
    html = requests.get(url)
    with open('file.txt', 'wb') as wr: 
        wr.write(html.content)
    wr.close()

    read = open("file.txt","r+",encoding="utf-8")

    listhtmlExcel = []
    htmlExcel = ""
    for line in read:
        xls = ".xls"
        if xls in line:
            htmlExcel = line
            listhtmlExcel.append(htmlExcel)
    
    listaExcel = []
    for j in range(0 , len(listhtmlExcel)):
        htmlExcel = listhtmlExcel[j]
        htmlExcel.replace(">", " ")
        htmlExcel.replace("<", " ")
        

        separacionExcel= np.array(htmlExcel.split(" "))
        for i in range(0,len(separacionExcel)):
            if "href" in separacionExcel[i]:
                l = separacionExcel[i].split("\"")
                urlgit = "https://github.com"+l[1]
                urlgit2 = urlgit.split("blob")
                urlgit = urlgit2[0]+"raw"+urlgit2[1]
                listaExcel.append(urlgit)
                
    read.close()
    listaExcel.append(" ")
    return listaExcel



In [11]:
def RunFile():
    print("WELCOME!")
    print("BEFORE STARTING CHECK YOU DONT HAVE ANY EXCEL FILES IN THIS FOLDER, ONCE YOU ARE READY CLICK ENTER")
    x = input()
    print("FIRST OF ALL LETS START WITH THE ANH FILES FROM THE WEBPAGE")
    listaExcel = getFilesNames()
    downloadFiles (listaExcel)
    listnameGas, listnameOil, listnameAll = getGasOilFileNames(True)
    print(".....................")
    print("CREATING THE DATABASE")
    print(".....................")
    create_tables()
    guardarDatosOil(listnameOil)
    insertarBordeANH()
    print("FINISH DOWNLOADING ALL THE ANH FILES IN THE DATABASE")
    print("------------------------------------------------")
    print("DELETING THE EXCEL FILES")
    deleteFiles(listnameAll)
    print("------------------------------------------------")
    print("DOWNLOADING BLIND FILES FROM GITHUB:")
    namefilesGithub = getFilesNamesGitHub()
    downloadFiles(namefilesGithub)
    result = checkifwork()
    listnameGas, listnameOil, listnameAll = getGasOilFileNames(result)
     
    guardarDatosOil(listnameOil)
    print("THANK YOU FOR WAITING!")
    print("WE ARE DONE PLEASE RUN THE OTHER PYTHON FILES TO SEE THE RESULTS THANK YOU")
RunFile()

WELCOME!
BEFORE STARTING CHECK YOU DONT HAVE ANY EXCEL FILES IN THIS FOLDER, ONCE YOU ARE READY CLICK ENTER

FIRST OF ALL LETS START WITH THE ANH FILES FROM THE WEBPAGE
..................................................
Getting the name of the ANH files from the webpage
..................................................
downloading files... this might take some minutes
Download complete!
.....................
CREATING THE DATABASE
.....................
starting to save the data in the database this might take some minutes
saving:  Produccion-20Fiscalizada-20Crudo-2015.xlsx
finish with:  Produccion-20Fiscalizada-20Crudo-2015.xlsx
saving:  Produccion-20Fiscalizada-20Crudo-2018.xlsx
finish with:  Produccion-20Fiscalizada-20Crudo-2018.xlsx
saving:  Produccion-20Fiscalizada-20Crudo-2019-DIC.xlsx
finish with:  Produccion-20Fiscalizada-20Crudo-2019-DIC.xlsx
saving:  Produccion-20Fiscalizada-20Crudo-2020-20Agosto.xlsx
finish with:  Produccion-20Fiscalizada-20Crudo-2020-20Agosto.xlsx
saving:  P

downloading files... this might take some minutes
Download complete!
5 s
please enter the number of Excelfiles there are in github, in order to check and save in the database
5
the download was a success


In [3]:
deleteTable()